In [27]:
import pandas as pd
import csv
import requests
from scipy.stats import poisson
from bs4 import BeautifulSoup

In [8]:
clean = []

with open("./data/WorldCupMatches.csv", newline="") as f:
    reader = csv.reader(f)
    for split in reader:

        year = split[0]
        stage = split[2]
        home = split[5]
        home_goals = split[6]
        away_goals = split[7]
        away = split[8]
        away_team = split[-1]
        home_team = split[-2]

        strip = lambda x: x.strip()
        line = map(strip, [year, stage, home, home_goals, away_goals, away, away_team, home_team])
        clean.append(line)

with open("./clean/WorldCupMatchesClean.csv", "w+", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(clean)

In [38]:
all_world_cup_matches_data = pd.read_csv('./clean/WorldCupMatchesClean.csv')
home_games = all_world_cup_matches_data[['Home Team Initials', 'Home Team Goals', 'Away Team Goals']]
away_games = all_world_cup_matches_data[['Away Team Initials', 'Home Team Goals', 'Away Team Goals']]
home_games = home_games.rename(columns={'Home Team Initials':'Team Initials', 'Home Team Goals': 'GoalsScored', 'Away Team Goals': 'GoalsConceded'})
away_games = away_games.rename(columns={'Away Team Initials':'Team Initials', 'Home Team Goals': 'GoalsConceded', 'Away Team Goals': 'GoalsScored'})

team_goal_ratio = pd.concat([home_games, away_games], ignore_index=True).groupby(['Team Initials']).mean()
team_goal_ratio


GoalsScored  GoalsConceded
Team Initials                            
ALG               1.000000       1.500000
ANG               0.333333       0.666667
ARG               1.641975       1.049383
AUS               0.846154       2.000000
AUT               1.482759       1.620690
...                    ...            ...
URU               1.538462       1.403846
USA               1.117647       1.882353
WAL               0.800000       0.800000
YUG               1.621622       1.243243
ZAI               0.000000       4.666667

[82 rows x 2 columns]

In [189]:
def predict_points(home, away):
    if home in team_goal_ratio.index and away in team_goal_ratio.index:
        # goals_scored * goals_conceded
        lamb_home = team_goal_ratio.at[home,'GoalsScored'] * team_goal_ratio.at[away,'GoalsConceded']
        lamb_away = team_goal_ratio.at[away,'GoalsScored'] * team_goal_ratio.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [190]:
years = [1998, 2002, 2006, 2010, 2014, 2018]
import collections
collections.Callable = collections.abc.Callable

def get_fixtures(year):
    scrape = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(scrape)
    content = response.text
    parse = BeautifulSoup(content, 'lxml')
    fixtures = parse.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in fixtures:
        home.append(match.find('th', class_='fhome').get_text())
        """ score.append(match.find('th', class_='fscore').get_text()) """
        away.append(match.find('th', class_='faway').get_text())
   
    display = {'home': home, """ 'score': score, """ 'away': away}
    WCFrame = pd.DataFrame(display)
    WCFrame['year'] = year
    return WCFrame
  
FixtureFrame = get_fixtures(2022)
FixtureFrame.to_csv('./clean/fifa_worldcup_fixture.csv', index=False)

In [191]:
#create a dictionary with couuntry to respective code by scraping wikipedia
scrape = 'https://en.wikipedia.org/wiki/List_of_FIFA_country_codes'
response = requests.get(scrape)
content = response.text
parse = BeautifulSoup(content, 'lxml')
#get first four wikitables
tables = parse.find_all('table', class_='wikitable')
#keep only first four
tables = tables[:4]
#parse the four tables
country_code = []
country_name = []
for table in tables:
    for row in table.find_all('tr'):
        if row.find('td') is not None:
            country_name.append(row.find_all('td')[0].get_text().strip())
            country_code.append(row.find_all('td')[1].get_text().strip())
#create a dictionary with country to respective code
country_code_dict = dict(zip(country_name, country_code))
print(country_code_dict)

{'Afghanistan': 'AFG', 'Albania': 'ALB', 'Algeria': 'ALG', 'American Samoa': 'ASA', 'Andorra': 'AND', 'Angola': 'ANG', 'Anguilla': 'AIA', 'Antigua and Barbuda': 'ATG', 'Argentina': 'ARG', 'Armenia': 'ARM', 'Aruba': 'ARU', 'Australia': 'AUS', 'Austria': 'AUT', 'Azerbaijan': 'AZE', 'Bahamas': 'BAH', 'Bahrain': 'BHR', 'Bangladesh': 'BAN', 'Barbados': 'BRB', 'Belarus': 'BLR', 'Belgium': 'BEL', 'Belize': 'BLZ', 'Benin': 'BEN', 'Bermuda': 'BER', 'Bhutan': 'BHU', 'Bolivia': 'BOL', 'Bosnia and Herzegovina': 'BIH', 'Botswana': 'BOT', 'Brazil': 'BRA', 'British Virgin Islands': 'VGB', 'Brunei': 'BRU', 'Bulgaria': 'BUL', 'Burkina Faso': 'BFA', 'Burundi': 'BDI', 'Cambodia': 'CAM', 'Cameroon': 'CMR', 'Canada': 'CAN', 'Cape Verde': 'CPV', 'Cayman Islands': 'CAY', 'Central African Republic': 'CTA', 'Chad': 'CHA', 'Chile': 'CHI', 'China': 'CHN', 'Chinese Taipei': 'TPE', 'Colombia': 'COL', 'Comoros': 'COM', 'Congo': 'CGO', 'Cook Islands': 'COK', 'Costa Rica': 'CRC', 'Croatia': 'CRO', 'Cuba': 'CUB', 'Cur

In [192]:
df_fixture = pd.read_csv('./clean/fifa_worldcup_fixture.csv')
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()
df_fixture_knockout

home  'score': score, away  year
48  Netherlands          United States  2022
49    Argentina              Australia  2022
50       France                 Poland  2022
51      England                Senegal  2022
52        Japan                Croatia  2022
53       Brazil            South Korea  2022
54      Morocco                  Spain  2022
55     Portugal            Switzerland  2022

In [193]:
#group stages
#print(df_fixture_group_48)
#get home and away name of each fixture, convert the name into respective code, then use the respective code to calculate point, then add the point to the dataframe
for index, row in df_fixture_group_48.iterrows():
    if index == 0: continue
    print(row)
    home = row['home']
    away = row['away']
    print(home, away)
    home_code = country_code_dict[home.strip()]
    away_code = country_code_dict[away.strip()]
    home_point, away_point = predict_points(home_code, away_code)
    print(home, home_code, home_point, away, away_code, away_point)
    df_fixture_group_48.at[index, 'home_point'] = home_point
    df_fixture_group_48.at[index, 'away_point'] = away_point
print(df_fixture_group_48)
#write to new csv 
df_fixture_group_48.to_csv('./clean/fifa_worldcup_fixture_group_48.csv', index=False)


home                         Senegal 
 'score': score, away     Netherlands
year                             2022
Name: 1, dtype: object


KeyError: 'away'

In [194]:
#read the csv file fifa_worldcup_fixture_group_48.csv by 6 lines at a time and omit the first, calculate the top two teams in each of the 6 lines
df_fixture_group_48 = pd.read_csv('./clean/fifa_worldcup_fixture_group_48.csv')
df_fixture_group_48 = df_fixture_group_48[0:]

#iterate through each 6 lines
for i in range(0, len(df_fixture_group_48), 6):
    #get the 6 lines
    df = df_fixture_group_48[i:i+6]
    #go through all rows amd strip the whitespace
    for index, row in df.iterrows():
        df.at[index, 'home'] = row['home'].strip()
        df.at[index, 'away'] = row['away'].strip()
    
    #go through the home and away teams and calculate the total points and group by same team
    ndf = df.groupby(['home'])['home_point'].sum()
    #print(ndf)
    tdf = df.groupby(['away'])['away_point'].sum()
    #print(tdf)
    #add the two dataframes together and combine same team
    df = ndf.add(tdf, fill_value=0)
    #print(df)
    #sort the dataframe by points
    df = df.sort_values(ascending=False)
    print(df)
    #print("---")
    #get the top two teams
    df = df[:2]

home
Netherlands    3.871166
Senegal        2.533321
Ecuador        1.905392
Qatar          0.000000
dtype: float64
home
England          6.294365
Wales            5.024133
United States    3.523806
Iran             1.764762
dtype: float64
home
Argentina       6.881878
Poland          5.671591
Mexico          3.513837
Saudi Arabia    1.030282
dtype: float64
home
France       6.759452
Denmark      5.711685
Tunisia      2.506488
Australia    1.997231
dtype: float64
home
Germany       6.896990
Spain         4.981751
Costa Rica    2.706606
Japan         2.284659
dtype: float64
home
Croatia    6.561108
Belgium    5.257286
Morocco    4.608518
Canada     0.450222
dtype: float64
home
Brazil         7.462137
Switzerland    3.908147
Serbia         3.505039
Cameroon       1.966726
dtype: float64
home
Portugal       6.301379
Uruguay        5.125301
Ghana          3.863955
South Korea    1.651016
dtype: float64


In [195]:
matches = [["a1", "b2"], ["c1", "d2"], ["e1", "f2"], ["g1", "h2"], ["b1", "a2"], ["d1", "c2"], ["f1", "e2"], ["h1", "g2"]]
match_aligment = {

}

In [196]:
#knockout stages - round of 16
#parse fifa_worldcup_fixture_group_48_top2.csv and assign countries line by line to the matches
df_fixture_group_48_top2 = pd.read_csv('./clean/fifa_worldcup_fixture_group_48_top2.csv')
#iterate through each 2 lines
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
current_number = 1
current_letter = 0
#iterate through every linee
for i in range(0, len(df_fixture_group_48_top2)):
    #get country name, which is first column
    country = df_fixture_group_48_top2.iloc[i, 0]
    #print(country)
    #add to match alignment
    match_aligment[alphabet[current_letter]+str(current_number)] = country
    if current_number == 1: current_number = 2
    else: 
        current_number = 1
        current_letter += 1

In [202]:
#go through the matches and predict the winner between the two
winners = []
for match in matches:
    country_one = match[0]
    country_two = match[1]
    #add this to df_fixture_knockout
    #df_fixture_knockout = df_fixture_knockout.append({'home': country_one, 'away': country_two}, ignore_index=True)
    #print(match_aligment[country_one], match_aligment[country_two])

    #calculate winner 
    country_one_code = country_code_dict[match_aligment[country_one].strip()]
    country_two_code = country_code_dict[match_aligment[country_two].strip()]
    country_one_point, country_two_point = predict_points(country_one_code, country_two_code)
    #print(country_one_code, country_one_point, country_two_code, country_two_point)
    if country_one_point > country_two_point:
        #print(match_aligment[country_one], "wins")
        winners.append(match_aligment[country_one])
    else:
        #print(match_aligment[country_two], "wins")
        winners.append(match_aligment[country_two])
print(winners)
#write winners to csv
file = open('./clean/fifa_worldcup_fixture_knockout.csv', 'w')
writer = csv.writer(file)
writer.writerow(winners)

['Netherlands', 'Argentina', 'Germany', 'Brazil', 'England', 'France', 'Spain', 'Portugal']


68

In [204]:
#read fifa_worldcup_fixture_knockout.csv and read two lines at a time and put them against each other
df_fixture_knockout = pd.read_csv('./clean/fifa_worldcup_fixture_knockout.csv')
#print(df_fixture_knockout)
knockout_matches = [[1,2], [3,4], [5,6], [7,8]]
#iterate through each 2 lines
knockout_winners = []
for m in knockout_matches:
    country_one = df_fixture_knockout.columns[m[0]-1]
    country_two = df_fixture_knockout.columns[m[1]-1]
    #print(country_one, country_two)

    #calculate winner 
    country_one_code = country_code_dict[country_one.strip()]
    country_two_code = country_code_dict[country_two.strip()]
    country_one_point, country_two_point = predict_points(country_one_code, country_two_code)
    #print(country_one_code, country_one_point, country_two_code, country_two_point)
    if country_one_point > country_two_point:
        #print(country_one, "wins")
        knockout_winners.append(country_one)
    else:
        #print(country_two, "wins")
        knockout_winners.append(country_two)
print(knockout_winners)

['Netherlands', 'Germany', 'France', 'Portugal']


In [205]:
semi_final_matches = [[1,2], [3,4]]
semi_final_winners = []
for m in semi_final_matches:
    country_one = knockout_winners[m[0]-1]
    country_two = knockout_winners[m[1]-1]
    #print(country_one, country_two)

    #calculate winner 
    country_one_code = country_code_dict[country_one.strip()]
    country_two_code = country_code_dict[country_two.strip()]
    country_one_point, country_two_point = predict_points(country_one_code, country_two_code)
    #print(country_one_code, country_one_point, country_two_code, country_two_point)
    if country_one_point > country_two_point:
        #print(country_one, "wins")
        semi_final_winners.append(country_one)
    else:
        #print(country_two, "wins")
        semi_final_winners.append(country_two)
print(semi_final_winners)

['Germany', 'France']


In [213]:
country_one_code = country_code_dict[semi_final_winners[0].strip()]
country_two_code = country_code_dict[semi_final_winners[1].strip()]
country_one_point, country_two_point = predict_points(country_one_code, country_two_code)
print(country_one_code, country_one_point, country_two_code, country_two_point)
if country_one_point > country_two_point:
    print(semi_final_winners[0], "wins")
else:
    print(semi_final_winners[1], "wins")


GER 1.8802248346211758 FRA 0.9341288855184756
Germany wins
